<a href="https://colab.research.google.com/github/romirahmanw112/BigData2/blob/main/Praktek_big_data_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark -q


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Latihan_MLlib") \
    .getOrCreate()

print("Spark Session berhasil dibuat!")


Spark Session berhasil dibuat!


In [3]:
data_gaji = [
    (1.0, 20, 5000),
    (2.0, 22, 6000),
    (3.0, 25, 7000),
    (4.0, 26, 8500),
    (5.0, 30, 10000),
    (6.0, 31, 11500)
]

columns = ["pengalaman", "umur", "gaji"]
df_regresi = spark.createDataFrame(data_gaji, columns)

df_regresi.show()


+----------+----+-----+
|pengalaman|umur| gaji|
+----------+----+-----+
|       1.0|  20| 5000|
|       2.0|  22| 6000|
|       3.0|  25| 7000|
|       4.0|  26| 8500|
|       5.0|  30|10000|
|       6.0|  31|11500|
+----------+----+-----+



In [12]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["pengalaman", "umur"],
    outputCol="features"
)

data_siap_reg = assembler.transform(df_regresi) \
    .select("features", "gaji")

data_siap_reg.show(truncate=False)


+----------+-----+
|features  |gaji |
+----------+-----+
|[1.0,20.0]|5000 |
|[2.0,22.0]|6000 |
|[3.0,25.0]|7000 |
|[4.0,26.0]|8500 |
|[5.0,30.0]|10000|
|[6.0,31.0]|11500|
+----------+-----+



In [13]:
from pyspark.ml.regression import LinearRegression

# Bagi data
train_data, test_data = data_siap_reg.randomSplit([0.7, 0.3], seed=42)

# Model
lr = LinearRegression(featuresCol="features", labelCol="gaji")

# Training
model_lr = lr.fit(train_data)

# Prediksi
hasil_prediksi = model_lr.transform(test_data)

hasil_prediksi.select("features", "gaji", "prediction").show()


+----------+----+------------------+
|  features|gaji|        prediction|
+----------+----+------------------+
|[3.0,25.0]|7000|7439.3939393939345|
|[4.0,26.0]|8500| 8848.484848484854|
+----------+----+------------------+



In [14]:
print("Koefisien:", model_lr.coefficients)
print("Intercept:", model_lr.intercept)


Koefisien: [1484.8484848485002,-75.75757575758196]
Intercept: 4878.787878787984


In [15]:
data_churn = [
    (2.0, 5, 1),
    (1.0, 4, 1),
    (10.0, 0, 0),
    (12.0, 1, 0),
    (3.0, 3, 1),
    (15.0, 0, 0)
]

df_churn = spark.createDataFrame(
    data_churn, ["durasi", "komplain", "label"]
)

df_churn.show()


+------+--------+-----+
|durasi|komplain|label|
+------+--------+-----+
|   2.0|       5|    1|
|   1.0|       4|    1|
|  10.0|       0|    0|
|  12.0|       1|    0|
|   3.0|       3|    1|
|  15.0|       0|    0|
+------+--------+-----+



In [16]:
assembler_churn = VectorAssembler(
    inputCols=["durasi", "komplain"],
    outputCol="features"
)

data_siap_class = assembler_churn.transform(df_churn) \
    .select("features", "label")


In [17]:
from pyspark.ml.classification import LogisticRegression

log_reg = LogisticRegression(
    featuresCol="features",
    labelCol="label"
)

model_churn = log_reg.fit(data_siap_class)

hasil_churn = model_churn.transform(data_siap_class)

hasil_churn.select(
    "features", "label", "prediction", "probability"
).show(truncate=False)


+----------+-----+----------+-------------------------------------------+
|features  |label|prediction|probability                                |
+----------+-----+----------+-------------------------------------------+
|[2.0,5.0] |1    |1.0       |[2.7486447839081227E-14,0.9999999999999725]|
|[1.0,4.0] |1    |1.0       |[4.729924826979886E-13,0.999999999999527]  |
|[10.0,0.0]|0    |0.0       |[0.999999988084208,1.1915792019756566E-8]  |
|[12.0,1.0]|0    |0.0       |[0.9999999863708826,1.3629117390223655E-8] |
|[3.0,3.0] |1    |1.0       |[2.771802222998734E-8,0.9999999722819778]  |
|[15.0,0.0]|0    |0.0       |[0.9999999999999845,1.554312234475219E-14] |
+----------+-----+----------+-------------------------------------------+



In [19]:
data_mall = [
    (15000, 39),
    (12000, 81),
    (20000, 6),
    (25000, 77),
    (30000, 40)
]

df_mall = spark.createDataFrame(
    data_mall, ["pendapatan", "skor"]
)

df_mall.show()

+----------+----+
|pendapatan|skor|
+----------+----+
|     15000|  39|
|     12000|  81|
|     20000|   6|
|     25000|  77|
|     30000|  40|
+----------+----+



In [21]:
assembler_cluster = VectorAssembler(
    inputCols=["pendapatan", "skor"],
    outputCol="features"
)

data_siap_cluster = assembler_cluster.transform(df_mall)


In [22]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans().setK(3).setSeed(1)
model_km = kmeans.fit(data_siap_cluster)

prediksi_cluster = model_km.transform(data_siap_cluster)
prediksi_cluster.show()


+----------+----+--------------+----------+
|pendapatan|skor|      features|prediction|
+----------+----+--------------+----------+
|     15000|  39|[15000.0,39.0]|         1|
|     12000|  81|[12000.0,81.0]|         1|
|     20000|   6| [20000.0,6.0]|         2|
|     25000|  77|[25000.0,77.0]|         0|
|     30000|  40|[30000.0,40.0]|         0|
+----------+----+--------------+----------+



In [23]:
centers = model_km.clusterCenters()

for i, center in enumerate(centers):
    print(f"Cluster {i}: {center}")


Cluster 0: [27500.     58.5]
Cluster 1: [13500.    60.]
Cluster 2: [2.e+04 6.e+00]
